In [ ]:
from robot_properties_solo.solo12wrapper import Solo12Config
from src.scp_solver import solve_scp, interpolate_SCP_solution
from src.whole_body_control import WholeBodyModel
from src.centroidal_model import Centroidal_model
import config.conf_solo12_trot as conf
import matplotlib.pylab as plt
import pinocchio as pin
import numpy as np
import crocoddyl

In [ ]:
# Iteration 1: whole-body DDP (used to warm-start SCP)
# ---------------------------------------------------
# tracks a CoM forward direction based on the  
# number of foot steps.
# create and solve whole-body shooting problem 
print('running whole-Body DDP to warm start centroidal SCP ..', '\n')
wbd_model = WholeBodyModel(conf, TRACK_CENTROIDAL=False)
problem = wbd_model.createTrotShootingProblem()
solver = crocoddyl.SolverFDDP(problem)
xs = [wbd_model.rmodel.defaultState] * (solver.problem.T + 1)
us = solver.problem.quasiStatic([wbd_model.rmodel.defaultState] * solver.problem.T)
solver.solve(xs, us, 100, False, 0.1)
# save centroidal trajectories from whole-body solution
ddp_sol_1 = wbd_model.get_solution_trajectories(solver)
np.savez('wholeBody_to_centroidal_traj.npz', X=ddp_sol_1['centroidal'])

In [ ]:
#----------------------------------------------------------------------------
#                          NOMINAL TRAJ-OPT
# ---------------------------------------------------------------------------
# Iteration 2: centroidal SCP tracking centroidal states from DDP iteration 1
# ---------------------------------------------------------------------------
# create and solve centroidal scp problem
print('running centroidal SCP ..', '\n')
model = Centroidal_model(conf) 
# save centroidal trajectories from whole-body solution
scp_sol = solve_scp(model, conf.scp_params) 
scp_sol_interpol_nom = interpolate_SCP_solution(scp_sol)
np.savez('scp_sol_interpol_nom.npz', X=scp_sol_interpol_nom['X'], U=scp_sol_interpol_nom['U'])                
np.savez('centroidal_to_wholeBody_traj.npz', X=scp_sol['state'][-1], U=scp_sol['control'][-1])  

In [ ]:
# -----------------------------------------------------------------------------
# Iteration 3: whole-body DDP tracking centroidal SCP states and contact forces
# -----------------------------------------------------------------------------
# create and solve whole-body shooting problem
print('running whole-Body DDP to track centorial SCP ..', '\n')
wbd_model = WholeBodyModel(conf, TRACK_CENTROIDAL=True)
problem = wbd_model.createTrotShootingProblem()
solver = crocoddyl.SolverFDDP(problem)
xs = [wbd_model.rmodel.defaultState] * (solver.problem.T + 1)
us = solver.problem.quasiStatic([wbd_model.rmodel.defaultState] * solver.problem.T)
solver.setCallbacks([crocoddyl.CallbackLogger(),
                   crocoddyl.CallbackVerbose()])
solver.solve(xs, us, 100, False, 1)

In [ ]:
# --------------------------------------------------------
# Interpolate and save final solution for simulation later
# --------------------------------------------------------
# interpolate final whole-body solution 
ddp_sol_2 = wbd_model.get_solution_trajectories(solver)
ddp_interpolated_solution_nom = wbd_model.interpolate_whole_body_solution(ddp_sol_2)
np.savez('wholeBody_interpolated_traj.npz', X=ddp_interpolated_solution_nom['centroidal'], 
                                          U=ddp_interpolated_solution_nom['jointTorques'],
                                          q=ddp_interpolated_solution_nom['jointPos'],
                                          qdot=ddp_interpolated_solution_nom['jointVel'],
                                          gains=ddp_interpolated_solution_nom['gains'])

# get predicted contact positions, forces and jacobians from the last iteration of whole-body DDP
contact_positions_nom, ddp_forces_nom, contact_jacobians_nom = \
                wbd_model.get_contact_positions_and_forces_solution(solver)

In [ ]:
import sys
robot = Solo12Config.buildRobotWrapper()
# display whole-body motion in meshcat viewer
if conf.WITH_MESHCAT_DISPLAY:
    viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model)
    try:
        viz.initViewer(open=True)
    except ImportError as err:
        print(
            "Error while initializing the viewer. Make sure you installed Python meshcat"
        )
        print(err)
        sys.exit(0)
    viz.loadViewerModel()        
    q_des = ddp_interpolated_solution_nom['jointPos']
    for time_idx in range(conf.N_ctrl):
        viz.display(q_des[time_idx])

In [ ]:
#---------------------------------------------------------------------------------------
#                               STOCHASTIC TRAJ-OPT
# --------------------------------------------------------------------------------------
# Iteration 2: stochastic centroidal SCP tracking centroidal states from DDP iteration 1
# --------------------------------------------------------------------------------------
# create and solve centroidal scp problem
print('running stochastic centroidal SCP ..', '\n')
model = Centroidal_model(conf, STOCHASTIC_OCP=True) 
# save centroidal trajectories from whole-body solution
scp_sol_stoch = solve_scp(model, conf.scp_params)         
scp_sol_interpol_stoch = interpolate_SCP_solution(scp_sol)
np.savez('scp_sol_interpol_stoch.npz', X=scp_sol_interpol_stoch['X'], U=scp_sol_interpol_stoch['U'])              
np.savez('centroidal_to_wholeBody_traj.npz', X=scp_sol_stoch['state'][-1], U=scp_sol_stoch['control'][-1])       

In [ ]:
# ---------------------------------------------------------------------------------------
# Iteration 3: whole-body DDP tracking stochastic centroidal SCP states and contact forces
# ----------------------------------------------------------------------------------------
# create and solve whole-body shooting problem
print('running whole-Body DDP to track stochastic centorial SCP ..', '\n')
wbd_model = WholeBodyModel(conf, TRACK_CENTROIDAL=True)
problem = wbd_model.createTrotShootingProblem()
solver = crocoddyl.SolverFDDP(problem)
xs = [wbd_model.rmodel.defaultState] * (solver.problem.T + 1)
us = solver.problem.quasiStatic([wbd_model.rmodel.defaultState] * solver.problem.T)
solver.setCallbacks([crocoddyl.CallbackLogger(),
                crocoddyl.CallbackVerbose()])
solver.solve(xs, us, 100, False, 1)

In [ ]:
# --------------------------------------------------------
# Interpolate and save final solution for simulation later
# --------------------------------------------------------
# interpolate final whole-body solution and save in dat files 
ddp_sol_2 = wbd_model.get_solution_trajectories(solver)
ddp_interpolated_solution_stoch = wbd_model.interpolate_whole_body_solution(ddp_sol_2)
np.savez('wholeBody_interpolated_traj_stoch.npz', X=ddp_interpolated_solution_stoch['centroidal'], 
                                          U=ddp_interpolated_solution_stoch['jointTorques'],
                                          q=ddp_interpolated_solution_stoch['jointPos'],
                                          qdot=ddp_interpolated_solution_stoch['jointVel'],
                                          gains=ddp_interpolated_solution_stoch['gains'])
# get predicted contact positions, forces and jacobians from the last iteration of whole-body DDP
contact_positions_stoch, ddp_forces_stoch, contact_jacobians_stoch = wbd_model.get_contact_positions_and_forces_solution(solver) 


In [ ]:
from src.utils import plot_centroidal_tracking_cost, plot_contact_slippage, plot_contact_forces
plot_contact_forces(conf, scp_sol, scp_sol_stoch) 

# ------------------------------------
# preview whole-body motion in meschat
# ------------------------------------
import sys
robot = Solo12Config.buildRobotWrapper()
# display whole-body motion in meshcat viewer
if conf.WITH_MESHCAT_DISPLAY:
    viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model)
    try:
        viz.initViewer(open=True)
    except ImportError as err:
        print(
            "Error while initializing the viewer. Make sure you installed Python meshcat"
        )
        print(err)
        sys.exit(0)
    viz.loadViewerModel()        
    q_des = ddp_interpolated_solution_stoch['jointPos']
    for time_idx in range(conf.N_ctrl):
        viz.display(q_des[time_idx])

In [ ]:
# --------------------------------------
# Monte-carlo whole-body sims in pybullet 
# --------------------------------------
import jax
import pybullet
import numpy as np
import matplotlib.pylab as plt
import config.conf_solo12_trot as conf 
from src.simulate_solo import Simulator
from src.centroidal_model import Centroidal_model
from bullet_utils.env import BulletEnvWithGround
from robot_properties_solo.solo12wrapper import Solo12Robot

if conf.WITH_PYBULLET_SIMULATION:
    # create pybullet environment
    env = BulletEnvWithGround()
    # load optimized nominal trajectories
    centroidal_des = np.load('wholeBody_interpolated_traj.npz')['X']
    des_traj_nom = np.load('wholeBody_interpolated_traj.npz')
    ddp_gains = np.load('wholeBody_interpolated_traj.npz')['gains']
    x_ref_nom = np.load('scp_sol_interpol_nom.npz')['X']
    x_ref_nom_final = x_ref_nom[:3, -1]
    # load optimized stochastic trajectories
    centroidal_des_stoch = np.load('wholeBody_interpolated_traj_stoch.npz')['X']
    des_traj_stoch = np.load('wholeBody_interpolated_traj_stoch.npz')
    x_ref_stoch = np.load('scp_sol_interpol_stoch.npz')['X']
    x_ref_stoch_final = x_ref_stoch[:3, -1]
    # get contact sequence
    model = Centroidal_model(conf)
    contact_sequence = model._contact_data['contacts_logic']
    # create a pybullet simulation environment and
    # sample pseudorandom force disturbances 
    nb_sims = 20
    init_force_seed_1 = jax.random.PRNGKey(np.random.randint(0, 1000))
    init_time_seed_1 = jax.random.PRNGKey(np.random.randint(0, 1000))
    init_force_seed_2 = jax.random.PRNGKey(np.random.randint(0, 1000))
    init_time_seed_2 = jax.random.PRNGKey(np.random.randint(0, 1000))
    simulator = Simulator(env, Solo12Robot(), conf, contact_sequence, ddp_gains)
    force_tilde_1 = simulator.sample_pseudorandom_force_uncertainties_total(init_time_seed_1, 
                                                                    init_force_seed_1, nb_sims) 
    force_tilde_2 = simulator.sample_pseudorandom_force_uncertainties_total(init_time_seed_2, 
                                                                    init_force_seed_2, nb_sims)                                                             
    # run monte-carlo simulations 
    pybullet.changeDynamics(env.objects[0], -1, lateralFriction=0.4)
    # simulate trotting motions
    data_nom_1 = simulator.run(des_traj_nom, x_ref_nom_final, 'TROT', nb_sims, force_tilde_1, 'NOMINAL')
    data_stoch_1 = simulator.run(des_traj_stoch, x_ref_stoch_final,'TROT', nb_sims, force_tilde_1, 'STOCHASTIC')
    data_nom_2 = simulator.run(des_traj_nom, x_ref_nom_final, 'TROT_ON_DEBRI', nb_sims, force_tilde_2, 'NOMINAL')
    
    data_stoch_2 = simulator.run(des_traj_stoch, x_ref_stoch_final, 'TROT_ON_DEBRI', nb_sims, force_tilde_2, 'STOCHASTIC')
    pybullet.disconnect()

In [ ]:
# plot stuff 
from src.utils import plot_centroidal_tracking_cost, plot_contact_slippage
N = x_ref_nom.shape[1]
Q = conf.state_cost_weights
plot_centroidal_tracking_cost(nb_sims, Q, conf.dt_ctrl, x_ref_nom, x_ref_stoch, data_nom_1, data_nom_2, data_stoch_1, data_stoch_2)
plot_contact_slippage(nb_sims, conf.dt_ctrl, data_nom_1, data_nom_2, data_stoch_1, data_stoch_2)                                                                                                                                                                                
plt.show()    